# Bert를 사용한 한글 단어열 분류

copy from https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/blob/master/7.PRETRAIN_METHOD/7.2.1.bert_finetune_NSMC.ipynb

# 필요 라이브러리 설치

In [2]:
!pip install transformers==3.0.2
!pip install sentencepiece

     |████████████████████████████████| 769 kB 4.3 MB/s 
     |████████████████████████████████| 3.0 MB 44.9 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 
     |████████████████████████████████| 1.2 MB 53.8 MB/s 


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

from transformers import BertTokenizer
from transformers import TFBertModel

import tensorflow as tf

In [4]:
#random seed 고정
tf.random.set_seed(1234)
np.random.seed(1234)

SEQ_LENGTH = 128
BERT_MODEL_NAME = 'bert-base-multilingual-cased'

# 데이터

## 데이터 다운로드

In [5]:
!wget https://github.com/dhrim/deep_learning_data/raw/master/movie_ratings.txt

--2021-11-17 12:27:57--  https://github.com/dhrim/deep_learning_data/raw/master/movie_ratings.txt
Resolving github.com (github.com)... 52.69.186.44
Connecting to github.com (github.com)|52.69.186.44|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dhrim/deep_learning_data/master/movie_ratings.txt [following]
--2021-11-17 12:27:57--  https://raw.githubusercontent.com/dhrim/deep_learning_data/master/movie_ratings.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19515078 (19M) [text/plain]
Saving to: ‘movie_ratings.txt’

movie_ratings.txt   100%[===================>]  18.61M  --.-KB/s    in 0.1s    

2021-11-17 12:27:59 (124 MB/s) - ‘movie_ratings.txt’ saved [19515078/19515078]



## 데이터 로딩

In [6]:
df = pd.read_table("movie_ratings.txt")

In [7]:
df.head()

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


## 데이터 섞기

In [8]:
df = df.sample(frac=1).reset_index(drop=True) 

df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


## 필요 입출력 값 준비

In [9]:
reviews = df.document.values.copy().astype(np.str)
labels = df.label.values.copy().astype(np.int)

In [10]:
print(reviews.shape)
print(labels.shape)

(200000,)
(200000,)


필요 시, 실습 시간 관계로 전체 20,000개 중에 일부 만 사용한다.

In [11]:
reviews = reviews[:10000]
labels = labels[:10000]

## 토큰나이저 생성

In [12]:
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME, do_lower_case=False, model_max_length=SEQ_LENGTH)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

In [13]:
encoded_tokens= tokenizer.encode("토크나이징이 잘 될까여?")
print(encoded_tokens)
print(tokenizer.decode(encoded_tokens))

[101, 9873, 20308, 16439, 10739, 119233, 10739, 9654, 9100, 118671, 29935, 136, 102]
[CLS] 토크나이징이 잘 될까여? [SEP]


In [14]:
tokenized = tokenizer("토크나이징이 잘 될까여?", max_length=20, padding='max_length')
print(tokenized.keys())
print(tokenized['input_ids'])
print(tokenized['attention_mask'])
print(tokenized['token_type_ids'])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
[101, 9873, 20308, 16439, 10739, 119233, 10739, 9654, 9100, 118671, 29935, 136, 102, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## x, y 생성


tokernizer 사용 중에 경고 메시지가 많이 뜬다. 억제한다.


In [15]:
import logging
logging.basicConfig(level=logging.ERROR)

In [16]:
def build_model_input(reviews):
  input_ids = []
  attention_masks = []
  token_type_ids = []

  for review in reviews:
    tokenized = tokenizer(review, max_length=SEQ_LENGTH, padding='max_length')
    # tokenized = {'input_ids': [101, ...], 'token_type_ids': [0, ...], 'attention_mask': [1, ...]}
    input_ids.append(tokenized['input_ids'][:SEQ_LENGTH]) # 버그인지 몰라도 SEQ_LENGTH이상이어도 더 크게 나온다.
    attention_masks.append(tokenized['attention_mask'][:SEQ_LENGTH])
    token_type_ids.append(tokenized['token_type_ids'][:SEQ_LENGTH])

  return (np.array(input_ids), np.array(attention_masks), np.array(token_type_ids))


In [17]:
x = build_model_input(reviews)
y = labels

In [18]:
print(x[0].shape)

(10000, 128)


## train/test 분리

In [19]:
def split_bert_data(x, y, test_ratio):
  split_index = int(len(y)*(1-test_ratio))
  train_x = (x[0][:split_index], x[1][:split_index], x[2][:split_index])
  test_x  = (x[0][split_index:], x[1][split_index:], x[2][split_index:])
  train_y, test_y = y[:split_index], y[split_index:]

  return (train_x, train_y), (test_x, test_y)

(train_x, train_y), (test_x, test_y) = split_bert_data(x, y, test_ratio=0.2)

# 학습

## 모델 생성

In [20]:
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.layers import Dense, Dropout

class TFBertClassifier(tf.keras.Model):
  def __init__(self):
    super(TFBertClassifier, self).__init__()

    self.bert = TFBertModel.from_pretrained(BERT_MODEL_NAME)
    self.dropout = Dropout(self.bert.config.hidden_dropout_prob)
    self.classifier = Dense(2, kernel_initializer=TruncatedNormal(self.bert.config.initializer_range), 
                            name="classifier", activation="softmax")

  def call(self, inputs, attention_mask=None, token_type_ids=None, training=True):

    outputs = self.bert(inputs, attention_mask=attention_mask, token_type_ids=token_type_ids)
    # outputs 값: # sequence_output, pooled_output, (hidden_states), (attentions)
    pooled_output = outputs[1] 
    v = self.dropout(pooled_output, training=training)
    out = self.classifier(v)

    return out

model = TFBertClassifier()


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

참고로 Bert의 default 설정은 다음과 같다.

In [21]:
print(model.bert.config)

BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 119547
}



In [22]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

optimizer = Adam(3e-5)
loss = SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])


## 학습 실행

In [23]:
history = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
225/225 [==============================] - 119s 487ms/step - loss: 0.5715 - accuracy: 0.6950 - val_loss: 0.4699 - val_accuracy: 0.7713
Epoch 2/5
225/225 [==============================] - 108s 480ms/step - loss: 0.3966 - accuracy: 0.8318 - val_loss: 0.4857 - val_accuracy: 0.7950
Epoch 3/5
225/225 [==============================] - 108s 480ms/step - loss: 0.2839 - accuracy: 0.8840 - val_loss: 0.4645 - val_accuracy: 0.7912
Epoch 4/5
225/225 [==============================] - 108s 480ms/step - loss: 0.1953 - accuracy: 0.9229 - val_loss: 0.5920 - val_accuracy: 0.7950
Epoch 5/5
225/225 [==============================] - 108s 480ms/step - loss: 0.1324 - accuracy: 0.9514 - val_loss: 0.5962 - val_accuracy: 0.8100


In [24]:
loss, acc = model.evaluate(test_x, test_y, batch_size=32)
print("loss =", loss)
print("acc =", acc)

63/63 [==============================] - 13s 161ms/step - loss: 0.6138 - accuracy: 0.8020
loss = 0.6137772798538208
acc = 0.8019999861717224


## 분류 실행

In [25]:
def do_classify(test_text):
  model_input = build_model_input([test_text])
  y_ = model.predict(model_input)
  predicted = "긍정" if y_[0][1]>0.5 else "부정"

  print(test_text, "-->", predicted, ",score :",y_[0][1])

do_classify("여운이 많이 남는 영화")
do_classify("여운이 많이 남는 영화. 스토리 전개는 뻔함.")
do_classify("여운이 많이 남는 영화. 스토리 전개는 뻔함. 시간 때우기 용")
do_classify("여운이 많이 남는 영화. 스토리 전개는 뻔함. 시간 때우기 용, 비추.")

여운이 많이 남는 영화 --> 긍정 ,score : 0.9990569
여운이 많이 남는 영화. 스토리 전개는 뻔함. --> 긍정 ,score : 0.9871044
여운이 많이 남는 영화. 스토리 전개는 뻔함. 시간 때우기 용 --> 긍정 ,score : 0.8279504
여운이 많이 남는 영화. 스토리 전개는 뻔함. 시간 때우기 용, 비추. --> 부정 ,score : 0.43225986


## 다른 형태의 모델 코드

In [26]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense, Flatten, Dropout, Lambda

input_ids = Input(shape=(SEQ_LENGTH,), dtype=tf.int32, name="input_ids")
attention_mask = Input(shape=(SEQ_LENGTH,), dtype=tf.int32, name="attention_mask")
token_type_ids = Input(shape=(SEQ_LENGTH,), dtype=tf.int32, name="token_type_ids")

bert_layer = TFBertModel.from_pretrained(BERT_MODEL_NAME)

outputs = bert_layer(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
pooled_output = outputs[1] 
v = Dropout(0.1)(pooled_output)
output = Dense(2, activation="softmax", kernel_initializer=TruncatedNormal(0.02), name="classifier")(v)

model = tf.keras.Model([input_ids, attention_mask, token_type_ids], output)


optimizer = Adam(3e-5)
loss = SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 128)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 token_type_ids (InputLayer)    [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model_1 (TFBertModel)  ((None, 128, 768),   177853440   ['input_ids[0][0]',              
                                 (None, 768))                     'attention_mask[0][0]',     

In [27]:
history = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_split=0.1)

Epoch 1/5
225/225 [==============================] - 118s 487ms/step - loss: 0.5712 - accuracy: 0.6899 - val_loss: 0.4916 - val_accuracy: 0.7500
Epoch 2/5
225/225 [==============================] - 108s 479ms/step - loss: 0.3969 - accuracy: 0.8235 - val_loss: 0.4731 - val_accuracy: 0.7862
Epoch 3/5
225/225 [==============================] - 108s 479ms/step - loss: 0.2805 - accuracy: 0.8849 - val_loss: 0.5569 - val_accuracy: 0.7887
Epoch 4/5
225/225 [==============================] - 108s 479ms/step - loss: 0.1828 - accuracy: 0.9261 - val_loss: 0.6247 - val_accuracy: 0.7937
Epoch 5/5
225/225 [==============================] - 108s 479ms/step - loss: 0.1249 - accuracy: 0.9518 - val_loss: 0.7464 - val_accuracy: 0.7887


In [28]:
def do_classify(test_text):
  model_input = build_model_input([test_text])
  y_ = model.predict(model_input)
  predicted = "긍정" if y_[0][1]>0.5 else "부정"

  print(test_text, "-->", predicted, ",score :",y_[0][1])

do_classify("여운이 많이 남는 영화")
do_classify("여운이 많이 남는 영화. 스토리 전개는 뻔함.")
do_classify("여운이 많이 남는 영화. 스토리 전개는 뻔함. 시간 때우기 용")
do_classify("여운이 많이 남는 영화. 스토리 전개는 뻔함. 시간 때우기 용, 비추.")

여운이 많이 남는 영화 --> 긍정 ,score : 0.9987092
여운이 많이 남는 영화. 스토리 전개는 뻔함. --> 긍정 ,score : 0.98977333
여운이 많이 남는 영화. 스토리 전개는 뻔함. 시간 때우기 용 --> 긍정 ,score : 0.82239354
여운이 많이 남는 영화. 스토리 전개는 뻔함. 시간 때우기 용, 비추. --> 긍정 ,score : 0.8989455
